# 0. 準備

[クイックスタート: Azure AI 検索でエージェント取得を使用する](https://learn.microsoft.com/ja-jp/azure/search/search-get-started-agentic-retrieval?tabs=search-perms%2Csearch-endpoint&pivots=programming-language-python)を主に参照

Python実行前に以下の設定を実施

1. Python環境
    1. WSL環境でUbuntu22.04でPython3.13.2を使用
    1. 以下のPythonパッケージをインストール
        - azure-search-documents = "11.7.0b1"
        - azure-identity = "1.25.1"
        - openai = "2.2.0"
        - aiohttp = "3.13.0"
        - ipykernel = "6.30.1"
        - requests = "2.32.5"
1. Azure 環境
    1. Azure AI Search
        1. Azure AI Searchリソースを Basic で作成
        1. Azure Portalからメニュー 設定 -> キー で APIアクセス制御を「両方」に設定
        1. Azure Portalからメニュー 設定 -> ID で システム割り当て済みマネージドIDをONに設定
        1. 実行ユーザに以下のロールを割り当て
            - Search Service サービス貢献者
            - 検索インデックス データ共同作成者
            - 検索インデックス データ閲覧者
    1. Azure AI Foundry
        1. Azure AI Foundryリソースを作成
        1. Azure Portalからメニュー アクセス制御(IAM) で ロール割り当て「Cognitive Services OpenAI User」を前ステップで作成したシステム割り当て済みマネージドIDに追加
        1. AI Foundry Portal から モデルgpt-5-miniおよびtext-embedding-3-largeをデプロイ

固定値定義

In [7]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

search_endpoint = "https://ais-rag-eus.search.windows.net"
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
aoai_endpoint = "https://aif-rag-jpe.openai.azure.com"
aoai_embedding_model = "text-embedding-3-large"
aoai_embedding_deployment = "text-embedding-3-large"
aoai_gpt_model = "gpt-5-mini"
aoai_gpt_deployment = "gpt-5-mini"
index_name = "earth-at-night"
knowledge_source_name = "earth-knowledge-source"
knowledge_agent_name = "earth-knowledge-agent"
search_api_version = "2025-08-01-preview"

# 1. インデックス定義
ベクトル項目を含む検索インデックス定義の作成

In [8]:
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters,
    SemanticSearch,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
)
from azure.search.documents.indexes import SearchIndexClient
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(
    credential, "https://cognitiveservices.azure.com/.default"
)
index = SearchIndex(
    name=index_name,
    fields=[
        SearchField(
            name="id",
            type="Edm.String",
            key=True,
            filterable=True,
            sortable=True,
            facetable=True,
        ),
        SearchField(
            name="page_chunk",
            type="Edm.String",
            filterable=False,
            sortable=False,
            facetable=False,
        ),
        SearchField(
            name="page_embedding_text_3_large",
            type="Collection(Edm.Single)",
            stored=False,
            vector_search_dimensions=3072,
            vector_search_profile_name="hnsw_text_3_large",
        ),
        SearchField(
            name="page_number",
            type="Edm.Int32",
            filterable=True,
            sortable=True,
            facetable=True,
        ),
    ],
    vector_search=VectorSearch(
        profiles=[
            VectorSearchProfile(
                name="hnsw_text_3_large",
                algorithm_configuration_name="alg",
                vectorizer_name="azure_openai_text_3_large",
            )
        ],
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=aoai_endpoint,
                    deployment_name=aoai_embedding_deployment,
                    model_name=aoai_embedding_model,
                ),
            )
        ],
    ),
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                prioritized_fields=SemanticPrioritizedFields(
                    content_fields=[SemanticField(field_name="page_chunk")]
                ),
            )
        ],
    ),
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully.")

Index 'earth-at-night' created or updated successfully.


# 2. インデックスにアップロード
GitHubから分散表現を含んだearth-at-nightのデータを取得し、インデックスにPush

In [20]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"
documents = requests.get(url).json()

with SearchIndexingBufferedSender(endpoint=search_endpoint, index_name=index_name, credential=credential) as client:
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}' successfully.")

Documents uploaded to index 'earth-at-night' successfully.


# 3. ナレッジ ソースを作成
インデックスに対してナレッジソース作成

In [10]:
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters
from azure.search.documents.indexes import SearchIndexClient

ks = SearchIndexKnowledgeSource(
    name=knowledge_source_name,

    # このDescriptionをAgentが見て判断
    description="Knowledge source for Earth at night data",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name=index_name,
        source_data_select="id,page_chunk,page_number",
    ),
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_knowledge_source(knowledge_source=ks, api_version=search_api_version)
print(f"Knowledge source '{knowledge_source_name}' created or updated successfully.")

Knowledge source 'earth-knowledge-source' created or updated successfully.


# 4. ナレッジ エージェント作成

ナレッジソースに対してナレッジエージェントの作成。maxOutputsizeの値を変更  
https://github.com/MicrosoftDocs/azure-ai-docs/blob/main/articles/search/agentic-retrieval-how-to-create-knowledge-base.md?plain=1#L322

In [ ]:
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeAgentOutputConfiguration, KnowledgeAgentOutputConfigurationModality
from azure.search.documents.indexes import SearchIndexClient

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=aoai_endpoint,
    deployment_name=aoai_gpt_deployment,
    model_name=aoai_gpt_model,
)

# 自然言語での回答をさせる設定
output_cfg = KnowledgeAgentOutputConfiguration(
    modality=KnowledgeAgentOutputConfigurationModality.ANSWER_SYNTHESIS,  # 変更することで検索のみにすること可
    include_activity=True,
)

agent = KnowledgeAgent(
    name=knowledge_agent_name,
    models=[KnowledgeAgentAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(
            name=knowledge_source_name,
            reranker_threshold=2.0, # Rerankerが2.0以下は除外
        )
    ],
    output_configuration=output_cfg,
    request_limits={"maxOutputsize": 10_000}  # デフォルトから変更
)

index_client = SearchIndexClient(endpoint=search_endpoint, credential=credential)
index_client.create_or_update_agent(agent, api_version=search_api_version)
print(f"Knowledge agent '{knowledge_agent_name}' created or updated successfully.")

Knowledge agent 'earth-knowledge-agent' created or updated successfully.


# 5. 取得パイプライン実行

取得パイプライン実行。パイプライン内では以下の動作が実行される。

1. ユーザーの情報ニーズを推測するために、会話全体を分析します。
1. 複合クエリを、焦点を絞ったサブクエリに分解します。
1. サブクエリをナレッジ ソースに対して並列で実行します。
1. セマンティック ランカーを使用して、結果を再ランク付けし、フィルター処理します。
1. 上位の結果を統合し、自然言語による回答を生成します。

In [36]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import (
    KnowledgeAgentRetrievalRequest,
    KnowledgeAgentMessage,
    KnowledgeAgentMessageTextContent,
    SearchIndexKnowledgeSourceParams,
)
import textwrap
import json

instructions = """
A Q&A agent that can answer questions about the Earth at night.
If you don't have the answer, respond with "I don't know".
"""

messages = [{"role": "system", "content": instructions}]

agent_client = KnowledgeAgentRetrievalClient(
    endpoint=search_endpoint, agent_name=knowledge_agent_name, credential=credential
)
# query_1 = """
#    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
#    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
#    """
query_1 = """
    なぜ郊外のベルト地帯は、絶対的な光量が都心部の方が高いにもかかわらず、12月の明るさの増加がより大きくなるのでしょうか？
    なぜフェニックスの夜間の街路網は宇宙からはっきりと見えるのに、米国中西部の都市間を結ぶ高速道路の広い区間は比較的暗いままなのでしょうか？
    """
messages.append({"role": "user", "content": query_1})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])],
        )
        for m in messages
        if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name, kind="searchIndex"
        )
    ],
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version,)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

print("----回答----")
print(textwrap.fill(result.response[0].content[0].text, width=120))

print("----Activity(回答までの処理)----")
for activity in result.activity:
    if hasattr(activity, "search_index_arguments"):
        print("search_index_param: " + str(activity.search_index_arguments))
    else:
        print(activity)

print("検索結果")
print(json.dumps([r.as_dict() for r in result.references], indent=2))

Retrieved content from 'earth-knowledge-source' successfully.
----回答----
情報は次のように見つかりました。郊外ベルト地帯の12月の明るさ増加が大きくなる主な理由は、庭や屋外飾りを設置しやすい一戸建て住宅が多く、郊外・周縁部でホリデー照明が劇的に増えるためであると報告されています（中心市街地は絶対的には明るいが、増
加率は小さいことが多い）[ref_id:2][ref_id:3]. また、米国全体では12月に夜間光が20～50％明るくなるといった観測もあります[ref_id:8][ref_id:5].
フェニックスの街路網が宇宙からはっきり見える理由としては、市街地が規則的な区画グリッドで舗装道路の街路灯が連続的に並び、主要幹線（たとえばGrand
Avenue）や商工業用施設沿いの強い照明ノードがあるため、夜間にその格子状パターンが低軌道から明瞭に現れることが挙げられます[ref_id:0][ref_id:4].  一方で、米国中西部の都市間を結ぶ高速道路の広い区間が比較的暗いままで
あるのは、それらの区間が人口希薄地帯や農地を横切り、長距離にわたって沿道開発や常時点灯する道路照明が少ないためであると示されています。夜間画像では人口密度や都市化の少ない西部・中部の地域は暗く映ることが指摘されています[ref_id:16]
[ref_id:13]. さらに、高速道路が明瞭に見えるのは、道路沿いに照明が新たに整備されたり（例：Jane Addams Memorial
Tollwayの照明強化が目立つケース）、周辺の開発が進んだ場合に限られることが観察されています[ref_id:15].
----Activity(回答までの処理)----
{'additional_properties': {}, 'id': 0, 'type': 'modelQueryPlanning', 'elapsed_ms': 21694, 'input_tokens': 2121, 'output_tokens': 1835}
search_index_param: {'additional_properties': {}, 'search': '郊外 ベルト地帯 12月 明るさ 増加 都心部 より大きい 理由 ホリデーライ

# 6. 会話を続ける

In [37]:
# query_2 = "How do I find lava at night?"
query_2 = "夜に溶岩を見つけるにはどうすればいいですか?"
messages.append({
    "role": "user",
    "content": query_2
})

req = KnowledgeAgentRetrievalRequest(
    messages=[
        KnowledgeAgentMessage(
            role=m["role"],
            content=[KnowledgeAgentMessageTextContent(text=m["content"])]
        ) for m in messages if m["role"] != "system"
    ],
    knowledge_source_params=[
        SearchIndexKnowledgeSourceParams(
            knowledge_source_name=knowledge_source_name,
            kind="searchIndex"
        )
    ]
)

result = agent_client.retrieve(retrieval_request=req, api_version=search_api_version)
print(f"Retrieved content from '{knowledge_source_name}' successfully.")

print("----回答----")
print(textwrap.fill(result.response[0].content[0].text, width=120))

print("----Activity(回答までの処理)----")
for activity in result.activity:
    if hasattr(activity, "search_index_arguments"):
        print("search_index_param: " + str(activity.search_index_arguments))
    else:
        print(activity)

print("検索結果")
print(json.dumps([r.as_dict() for r in result.references], indent=2))

Retrieved content from 'earth-knowledge-source' successfully.
----回答----
情報が見つかりました。夜間に溶岩を見つけるには、衛星の夜間可視データと熱赤外データを組み合わせて探すのが有効です [ref_id:1][ref_id:11].  1) VIIRS の Day/Night
Band（夜間可視）で赤〜橙色の光点や夜間の光輝（溶岩や火災として見える明るい点）を探す。VIIRS DNB は野火やガスフレアのような薄い光源を検出でき、夜間に流れる溶岩の光も可視化される例が報告されています（たとえばエトナの夜間画像）
[ref_id:1][ref_id:0][ref_id:5][ref_id:3].  2) 検出を確認するには熱赤外センサー（Landsat 8 の OLI/TIRS
など）で熱シグネチャを調べる。熱画像では高温の溶岩が明るく（非常に温かい領域として）示され、溶岩流や活動中の割れ目を明確に示すことができます（キラウエアの事例など） [ref_id:11][ref_id:14].  3)
宇宙飛行士の夜間写真や高解像度の夜間撮影も有用で、溶岩の赤橙色の光を直接とらえた写真例があります（ISS撮影のエトナや他の夜間撮影例） [ref_id:3][ref_id:2].  4)
時系列画像で推移を追うと、溶岩の進行や噴火継続を把握しやすくなります（Landsat の時系列で溶岩流の進行を追った例） [ref_id:14].  5) Black Marble
のような夜間合成や雲除去処理を用いて、月光や大気の影響を補正した画像で確認すると誤検出を減らせます（夜間光の補正・合成手法の説明） [ref_id:9][ref_id:18][ref_id:1].  まとめ：夜間可視（VIIRS
DNB）で光る点を見つけ、熱赤外（Landsat OLI/TIRS 等）で熱的に確認し、宇宙飛行士写真や時系列データ、夜間合成画像で経時的・空間的に検証するのが有効です
[ref_id:1][ref_id:0][ref_id:3][ref_id:11][ref_id:14][ref_id:9].
----Activity(回答までの処理)----
{'additional_properties': {}, 'id':